## Rethinking the value of Network pruning 실험

모델 : VGG16, ResNet50

Dataset : CIFAR100

Pruning Strategies : Predefined (L1 based pruning) / Automatic (Network slimming) / Unstructured 

Scenario : Scratch-B, FT 

Metrics : Accuracy, Model size, FLOPs , pruned ratio for each channel



# 필요한 library load

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchsummary import summary
import torch.nn.utils.prune as prune
from tqdm import tqdm
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Paths for saving and loading models
VGG16_MODEL_PATH = 'vgg_trained_model.pth'
RESNET50_MODEL_PATH = 'resnet_trained_model.pth'
PRUNED_MODEL_PATH = 'pruned_model.pth'

Using device: cuda


# CIFAR100 Dataset

In [2]:
# CIFAR-100 Dataset
def get_dataloaders(batch_size=128):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276)),
    ])
    train_set = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    test_set = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# VGG-16 모델 훈련 (22 epoch 수행)

In [3]:

# Train VGG-16 model from scratch
def train_vgg16(epochs=50, save_path=VGG16_MODEL_PATH):
    model = models.vgg16(num_classes=100).to(device)  # Send model to GPU
    train_loader, test_loader = get_dataloaders()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    # Training loop with tqdm progress bar
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        loop = tqdm(train_loader, total=len(train_loader), leave=False)
        for i, (inputs, labels) in enumerate(loop):
            inputs, labels = inputs.to(device), labels.to(device)  # Send inputs and labels to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Update tqdm loop
            loop.set_description(f'Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=running_loss/(i+1))

        # Perform validation after each epoch
        validate_model(model, test_loader, criterion)

    # Save the model after training
    torch.save(model.state_dict(), save_path)


# ResNet-50 훈련 

In [4]:

# Train ResNet-50 model from scratch
def train_resnet50(epochs=50, save_path=RESNET50_MODEL_PATH):
    model = models.resnet50(num_classes=100).to(device)  # Send model to GPU
    train_loader, test_loader = get_dataloaders()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    # Training loop with tqdm progress bar
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        loop = tqdm(train_loader, total=len(train_loader), leave=False)
        for i, (inputs, labels) in enumerate(loop):
            inputs, labels = inputs.to(device), labels.to(device)  # Send inputs and labels to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Update tqdm loop
            loop.set_description(f'Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=running_loss/(i+1))

        # Perform validation after each epoch
        validate_model(model, test_loader, criterion)

    # Save the model after training
    torch.save(model.state_dict(), save_path)

# Pruning 방법론 3가지
1. automatic
2. Predefined
3. Unstructured

In [5]:

# Pruning methods
def prune_model(model, strategy='automatic', pruning_ratio=0.5):
    if strategy == 'automatic':
        # Automatic (Unstructured) pruning of weights
        parameters_to_prune = []
        for name, module in model.named_modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                parameters_to_prune.append((module, 'weight'))
        
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method=prune.L1Unstructured,
            amount=pruning_ratio,
        )
    elif strategy == 'predefined':
        # Predefined (Structured) pruning of filters/channels or neurons
        for name, module in model.named_modules():
            if isinstance(module, nn.Conv2d):
                prune.ln_structured(module, name='weight', amount=pruning_ratio, n=2, dim=0)  # prune entire filters
            elif isinstance(module, nn.Linear):
                prune.ln_structured(module, name='weight', amount=pruning_ratio, n=2, dim=1)  # prune entire neurons
    elif strategy == 'unstructured':
        # Unstructured pruning - pruning individual weights
        for module in model.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                prune.l1_unstructured(module, name='weight', amount=pruning_ratio)

    return model

# Scratch-E 모델 (epoch를 같게 한 pruned 모델)

In [6]:
# Scratch-E: Train the pruned model from scratch with the same number of epochs as the original training
def train_scratch_e(model, epochs=50, pruning_ratio=0.5, strategy='automatic', save_path=PRUNED_MODEL_PATH):
    model = prune_model(model, strategy=strategy, pruning_ratio=pruning_ratio).to(device)  # Prune and move to GPU
    train_loader, test_loader = get_dataloaders()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        loop = tqdm(train_loader, total=len(train_loader), leave=False)
        for i, (inputs, labels) in enumerate(loop):
            inputs, labels = inputs.to(device), labels.to(device)  # Send inputs and labels to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Update tqdm loop
            loop.set_description(f'Scratch-E Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=running_loss/(i+1))

        # Perform validation after each epoch
        validate_model(model, test_loader, criterion)

    # Save the Scratch-E trained pruned model
    torch.save(model.state_dict(), save_path)


# FT Pruned model

In [7]:

# Fine-tune pruned model with progress bar
def fine_tune_model(model, epochs=3, save_path=PRUNED_MODEL_PATH):
    model = model.to(device)  # Ensure the model is on the GPU
    train_loader, test_loader = get_dataloaders()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        loop = tqdm(train_loader, total=len(train_loader), leave=False)
        for i, (inputs, labels) in enumerate(loop):
            inputs, labels = inputs.to(device), labels.to(device)  # Send inputs and labels to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Update tqdm loop
            loop.set_description(f'Fine-tuning Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=running_loss/(i+1))

    # Save the fine-tuned pruned model
    torch.save(model.state_dict(), save_path)

# Metrics for comparsion

In [13]:
# Validation function to return accuracy, precision, and recall
def validate_model(model, test_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    val_loss /= len(test_loader)
    accuracy = 100 * correct / total
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')

    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%, Precision: {precision:.2f}, Recall: {recall:.2f}')
    model.train()
    return accuracy, precision, recall

# Calculate and display additional metrics
def calculate_model_metrics(model, label=""):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    pruned_params = sum(torch.sum(p == 0).item() for p in model.parameters())
    pruned_ratio = pruned_params / total_params * 100

    print(f'{label} Model Summary:')
    print(f'  Total Parameters: {total_params}')
    print(f'  Pruned Parameters: {pruned_params}')
    print(f'  Pruned Ratio: {pruned_ratio:.2f}%\n')

    return total_params, pruned_params, pruned_ratio


In [11]:
# Evaluate the model on test data and print results
def test_model(model, test_loader, criterion, label=""):
    print(f"\n--- Testing {label} ---")
    accuracy, precision, recall = validate_model(model, test_loader, criterion)
    print(f'{label} Test Results:')
    print(f'  Accuracy: {accuracy:.2f}%')
    print(f'  Precision: {precision:.2f}')
    print(f'  Recall: {recall:.2f}\n')


In [14]:

# Main workflow
if __name__ == "__main__":
    # Load data
    train_loader, test_loader = get_dataloaders()

    # Step 1: Train VGG-16 from scratch
    print("Training VGG-16 from scratch...\n")
    vgg16_model = models.vgg16(num_classes=100).to(device)
    train_vgg16(epochs=50)
    
    # Display VGG-16 original model metrics
    print("\n--- VGG-16 Original Model ---")
    vgg16_acc, vgg16_precision, vgg16_recall = validate_model(vgg16_model, test_loader, nn.CrossEntropyLoss())
    calculate_model_metrics(vgg16_model, "VGG-16 Original")
    
    # Test the VGG-16 original model
    test_model(vgg16_model, test_loader, nn.CrossEntropyLoss(), "VGG-16 Original")

    # Step 2: Train ResNet-50 from scratch
    print("\nTraining ResNet-50 from scratch...\n")
    resnet_model = models.resnet50(num_classes=100).to(device)
    train_resnet50(epochs=50)

    # Display ResNet-50 original model metrics
    print("\n--- ResNet-50 Original Model ---")
    resnet_acc, resnet_precision, resnet_recall = validate_model(resnet_model, test_loader, nn.CrossEntropyLoss())
    calculate_model_metrics(resnet_model, "ResNet-50 Original")
    
    # Test the ResNet-50 original model
    test_model(resnet_model, test_loader, nn.CrossEntropyLoss(), "ResNet-50 Original")

    # Step 3: Apply Scratch-E (pruned and retrained) on VGG-16
    print("\nTraining VGG-16 Scratch-E (Pruned and retrained from scratch)...\n")
    vgg16_model_scratch_e = models.vgg16(num_classes=100).to(device)
    train_scratch_e(vgg16_model_scratch_e, epochs=50, pruning_ratio=0.5, strategy='automatic')
    
    # Display VGG-16 Scratch-E model metrics
    print("\n--- VGG-16 Scratch-E Model ---")
    vgg16_scratch_acc, vgg16_scratch_precision, vgg16_scratch_recall = validate_model(vgg16_model_scratch_e, test_loader, nn.CrossEntropyLoss())
    calculate_model_metrics(vgg16_model_scratch_e, "VGG-16 Scratch-E")
    
    # Test the VGG-16 Scratch-E model
    test_model(vgg16_model_scratch_e, test_loader, nn.CrossEntropyLoss(), "VGG-16 Scratch-E")

    # Step 4: Apply Scratch-E (pruned and retrained) on ResNet-50
    print("\nTraining ResNet-50 Scratch-E (Pruned and retrained from scratch)...\n")
    resnet_model_scratch_e = models.resnet50(num_classes=100).to(device)
    train_scratch_e(resnet_model_scratch_e, epochs=50, pruning_ratio=0.5, strategy='automatic')

    # Display ResNet-50 Scratch-E model metrics
    print("\n--- ResNet-50 Scratch-E Model ---")
    resnet_scratch_acc, resnet_scratch_precision, resnet_scratch_recall = validate_model(resnet_model_scratch_e, test_loader, nn.CrossEntropyLoss())
    calculate_model_metrics(resnet_model_scratch_e, "ResNet-50 Scratch-E")
    
    # Test the ResNet-50 Scratch-E model
    test_model(resnet_model_scratch_e, test_loader, nn.CrossEntropyLoss(), "ResNet-50 Scratch-E")

Files already downloaded and verified
Files already downloaded and verified
Training VGG-16 from scratch...

Files already downloaded and verified
Files already downloaded and verified


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Loss: 4.4475, Accuracy: 2.13%, Precision: 0.00, Recall: 0.02


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Loss: 4.2987, Accuracy: 4.24%, Precision: 0.01, Recall: 0.04


...

Validation Loss: 4.9177, Accuracy: 37.25%, Precision: 0.37, Recall: 0.37


Validation Loss: 4.9083, Accuracy: 37.65%, Precision: 0.37, Recall: 0.38


Validation Loss: 4.9392, Accuracy: 37.97%, Precision: 0.38, Recall: 0.38


Validation Loss: 4.9288, Accuracy: 37.80%, Precision: 0.37, Recall: 0.38


Validation Loss: 4.9190, Accuracy: 38.00%, Precision: 0.37, Recall: 0.38

--- ResNet-50 Scratch-E Model ---
Validation Loss: 4.9190, Accuracy: 38.00%, Precision: 0.37, Recall: 0.38
ResNet-50 Scratch-E Model Summary:
  Total Parameters: 23712932
  Pruned Parameters: 90
  Pruned Ratio: 0.00%


--- Testing ResNet-50 Scratch-E ---
Validation Loss: 4.9190, Accuracy: 38.00%, Precision: 0.37, Recall: 0.38
ResNet-50 Scratch-E Test Results:
  Accuracy: 38.00%
  Precision: 0.37
  Recall: 0.38

